### AMM Optimization, 1st model

In [1]:
using Gurobi, JuMP, CSV, Tables

In [ ]:
# Load data
data = CSV.read("experiment_1_liquidity.csv", Tables.matrix; header=0)

# Transform the data into a vector of liquidity values
liquidity = data[:, 2]

In [ ]:
# With an order amount of token x

function optimize_order_execution(amount_token_x_per_pool, amount_token_y_per_pool, liquidity, order_amount)
    # Create a new model
    model = Model(Gurobi.Optimizer)

    # Create variables
    @variable(model, 0 <= x[1:length(liquidity)] <= amount_token_x_per_pool)

    # Set objective
    @objective(model, Min, sum(amount_token_y_per_pool[i] / (amount_token_x_per_pool[i]+ x[i]) for i in 1:length(liquidity)))

    # Add constraint
    @constraint(model, sum(x) == order_amount)

    # Optimize model
    optimize!(model)

    # Return the optimal solution
    return value.(x)
end

x = optimize_order_execution(liquidity, liquidity, liquidity, 1000)